In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import ops

In [2]:
mnist = input_data.read_data_sets('/mnist/input_data', one_hot=True)

Extracting /mnist/input_data\train-images-idx3-ubyte.gz
Extracting /mnist/input_data\train-labels-idx1-ubyte.gz
Extracting /mnist/input_data\t10k-images-idx3-ubyte.gz
Extracting /mnist/input_data\t10k-labels-idx1-ubyte.gz


In [21]:
# training data
X_train = mnist.train.images.reshape([-1, 28, 28, 1])
Y_train = mnist.train.labels
print("Training: X shape: {}".format(X_train.shape))
print("Training: Y shape: {}".format(Y_train.shape))

# test data
# taking a subset of test data (my GPU goes OOM on 10K)
X_test,Y_test  = mnist.test.next_batch(5000)
X_test = X_test.reshape([-1,28,28,1])
print("Test: X shape: {}".format(X_test.shape))
print("Test: Y shape: {}".format(Y_test.shape))


Training: X shape: (55000, 28, 28, 1)
Training: Y shape: (55000, 10)
Test: X shape: (5000, 28, 28, 1)
Test: Y shape: (5000, 10)


In [26]:
def train(X_train, Y_train):
        
        ops.reset_default_graph()
        
        # Create placeholders
        (m,h,w,c) = X_train.shape
        y = Y_train.shape[1]
        
        X = tf.placeholder(shape=(None,h,w,c), dtype = tf.float32)
        Y = tf.placeholder(shape=(None,y), dtype=tf.float32)

       # Conv layer 1 with Max Pooling 
        filter = tf.get_variable("W1", [5, 5, 1, 32], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
        Z1 = tf.nn.conv2d(X, filter= filter, strides= [1,1,1,1], padding='SAME')
        A1 = tf.nn.relu(Z1)
        P1 = tf.nn.max_pool(A1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
        print(P1.shape)
        
        # Conv layer 2 with Max Pooling
        filter2 = tf.get_variable("W2", [5, 5, 32, 64], initializer = tf.contrib.layers.xavier_initializer(seed = 0))
        Z2 = tf.nn.conv2d(P1, filter=filter2, strides= [1,1,1,1], padding='SAME')
        A2 = tf.nn.relu(Z2)
        P2 = tf.nn.max_pool(A2, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
        print(P2.shape)
        
        # fully connected layer
        L = tf.contrib.layers.flatten(P2)
        Z = tf.contrib.layers.fully_connected(L,10,activation_fn=None)
        print(Z.shape)
        
        # compute cross entropy
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=Z))
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
                
        # calculate accuracy
        correct_prediction = tf.equal(tf.argmax(Z, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        init = tf.global_variables_initializer()

        with tf.Session() as sess:
            sess.run(init)
            for i in range(20000):
                batch = mnist.train.next_batch(64)
                
                if i % 500 == 0:
                    train_accuracy = accuracy.eval(feed_dict={X: batch[0].reshape([-1, 28, 28, 1]), Y: batch[1]})
                    print('step %d, training accuracy %g' % (i, train_accuracy))
                
                train_step.run(feed_dict={X: batch[0].reshape([-1, 28, 28, 1]), Y: batch[1]})

            print('test accuracy %g' % accuracy.eval(feed_dict={X: X_test, Y: Y_test}))
        return

In [27]:
train(X_train, Y_train)

(?, 14, 14, 32)
(?, 7, 7, 64)
(?, 10)
step 0, training accuracy 0.078125
step 500, training accuracy 0.890625
step 1000, training accuracy 0.9375
step 1500, training accuracy 0.953125
step 2000, training accuracy 0.953125
step 2500, training accuracy 0.96875
step 3000, training accuracy 0.953125
step 3500, training accuracy 0.984375
step 4000, training accuracy 0.96875
step 4500, training accuracy 0.96875
step 5000, training accuracy 1
step 5500, training accuracy 1
step 6000, training accuracy 0.984375
step 6500, training accuracy 0.984375
step 7000, training accuracy 0.984375
step 7500, training accuracy 0.96875
step 8000, training accuracy 0.96875
step 8500, training accuracy 0.96875
step 9000, training accuracy 1
step 9500, training accuracy 1
step 10000, training accuracy 1
step 10500, training accuracy 0.984375
step 11000, training accuracy 1
step 11500, training accuracy 1
step 12000, training accuracy 0.984375
step 12500, training accuracy 0.984375
step 13000, training accuracy